In [1]:
import re
import numpy as np
import scipy

def _compatibility_matrix(lines):
    """Compute a binary compatibility matrix with 1's around each symbol"""
    p1 = re.compile(r'[^0123456789.]')
    p2 = re.compile(r'[0123456789.]')
    p3 = re.compile(r'x')

    # Convert input to a binary array
    arr = np.zeros([len(lines), len(lines[0])]) 
    for i, line in enumerate(lines):
        line = re.sub(p1,'x',line)      # Change symbols to x
        line = re.sub(p2,'0',line)      # Change digits and periods to 0
        line = re.sub(p3, '1', line)    # Change x to 1
        arr[i:] = np.fromiter((int(c) for c in line), dtype = int)

    # Expand each 1 to its 8 neighbours
    mask = np.array([[1,1,1],
                     [1,1,1],
                     [1,1,1]])
    filtered_arr = scipy.signal.convolve2d(arr, mask, mode='same')
    
    # Threshold and convert to int - not really necssary but cleaner
    return np.where(filtered_arr>=1, 1, 0).astype(int)

def aoc23_3_proc_1(lines,verbose=False):
    sum = 0
    c_mat = _compatibility_matrix(lines)
    p = re.compile(r'(\d+)')
    for i, line in enumerate(lines):
        numbers = p.finditer(line)
        for n in numbers:
            # Check if number overlaps any non-zero items in the compatibility matrix
            start, end = n.span()
            slc = c_mat[i,start:end]
            if (np.sum(slc)>0):
                sum += int(n.group())
    return sum

In [2]:
with open('aoc23-3-input.txt', 'r') as file:
    lines = [s.strip() for s in file.readlines()]
sum = aoc23_3_proc_1(lines,False)
print(sum)

530495


In [15]:
def _find_adjacent(pos, m_num):
    """Compute a list of numbers that are adjacent to pos"""
    result = []
    for m in m_num:
        num_start, num_end, = m.span()
        if pos>=num_start-1 and pos<=num_end:
            result.append(int(m.group()))
    return result

def aoc23_3_proc_2(lines,verbose=False):
    sum = 0
    p_star = re.compile(r'\*')
    p_num = re.compile(r'(\d+)')
    parts_prev = []
    parts_this = []
    parts_next = [m for m in p_num.finditer(lines[0])]
    for i, line in enumerate(lines):
        # Find the part numbers on the previous, current and next line
        parts_prev = parts_this.copy()
        parts_this = parts_next.copy()
        parts_next = []
        if i<len(lines)-1:
            parts_next = [m for m in p_num.finditer(lines[i+1])]

        # For each star, find the matching part numbers
        match_stars = p_star.finditer(line)
        for ms in match_stars:
            pos, _ = ms.span()
            parts = _find_adjacent(pos, parts_prev + parts_this + parts_next)
            if (len(parts)==2): # Must have exactly two matching parts
                sum += parts[0] * parts[1]
            if verbose:
                print(f'line {i}: star={pos} parts: {parts}')
    return sum

In [16]:
with open('aoc23-3-input.txt', 'r') as file:
    lines = [s.strip() for s in file.readlines()]
sum = aoc23_3_proc_2(lines,True)
print(sum)

line 1: star=12 parts: [65]
line 1: star=33 parts: [998, 874]
line 1: star=46 parts: [453, 407]
line 1: star=84 parts: [845, 240]
line 1: star=89 parts: [773, 345]
line 1: star=102 parts: [105]
line 1: star=115 parts: [307, 298]
line 1: star=122 parts: [527, 164]
line 2: star=26 parts: [125, 593]
line 2: star=36 parts: [331, 516]
line 2: star=73 parts: [76, 196]
line 2: star=92 parts: [861, 377]
line 3: star=63 parts: [752, 70]
line 3: star=132 parts: [175, 839]
line 4: star=68 parts: [74, 899]
line 4: star=84 parts: [289, 418]
line 5: star=10 parts: [750, 468]
line 5: star=36 parts: [497, 366]
line 5: star=50 parts: [258]
line 5: star=74 parts: [745, 234]
line 6: star=27 parts: [283, 681]
line 6: star=61 parts: [797, 960]
line 7: star=91 parts: [686, 847]
line 8: star=22 parts: [513, 45]
line 8: star=53 parts: [969, 883]
line 8: star=81 parts: [276, 988]
line 9: star=11 parts: [835, 180]
line 9: star=61 parts: [444, 584]
line 9: star=101 parts: [595]
line 9: star=126 parts: [707, 646]